In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES']= '0'

In [2]:
from transformers import AutoModel, AutoTokenizer
import torch

In [4]:
model = AutoModel.from_pretrained('mesolitica/llama2-embedding-1b-8k', 
                                  trust_remote_code = True, torch_dtype = torch.float16,
                                 use_flash_attention_2=True)
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama2-embedding-1b-8k')

You are attempting to use Flash Attention 2.0 with a model initialized on CPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [5]:
_ = model.cuda()

In [6]:
!mkdir gov.my

In [ ]:
import json
import os
from tqdm import tqdm

with open('gov.my.jsonl') as fopen:
    for no, l in tqdm(enumerate(fopen)):
        filename = os.path.join('gov.my', f'{no}.json')
        if os.path.exists(filename):
            continue
        try:
            l = json.loads(l)
            padded = tokenizer([l], return_tensors = 'pt').to('cuda')
            v = model.encode(padded).cpu().detach().numpy()[0].tolist()
            data = {
                'text': l,
                'v': v,
            }
            with open(filename, 'w') as f:
                json.dump(data, f)
        except Exception as e:
            pass

5519it [32:40,  6.23it/s]